# LLM Evaluation with MLflow Example Notebook

In this notebook, we will demonstrate how to evaluate various LLMs and RAG systems with MLflow, leveraging simple metrics such as toxicity, as well as LLM-judged metrics such as relevance, and even custom LLM-judged metrics such as professionalism

In [14]:
import os

import mlflow
import openai
import pandas as pd
from dotenv import load_dotenv

## More Setup

In [4]:
load_dotenv()

print('OPENAI_API_KEY=' + os.getenv('OPENAI_API_KEY')[:20] + '...')

OPENAI_API_KEY=sk-proj-ZhcCqw0r70dP...


## Basic Question-Answering Evaluation

Create a test case of `inputs` that will be passed into the model and `ground_truth` which will be used to compare against the generated output from the model.

In [5]:
eval_df = pd.DataFrame(
    {
        "inputs": [
            "How does useEffect() work?",
            "What does the static keyword in a function mean?",
            "What does the 'finally' block in Python do?",
            "What is the difference between multiprocessing and multithreading?",
        ],
        "ground_truth": [
            "The useEffect() hook tells React that your component needs to do something after render. React will remember the function you passed (we’ll refer to it as our “effect”), and call it later after performing the DOM updates.",
            "Static members belongs to the class, rather than a specific instance. This means that only one instance of a static member exists, even if you create multiple objects of the class, or if you don't create any. It will be shared by all objects.",
            "'Finally' defines a block of code to run when the try... except...else block is final. The finally block will be executed no matter if the try block raises an error or not.",
            "Multithreading refers to the ability of a processor to execute multiple threads concurrently, where each thread runs a process. Whereas multiprocessing refers to the ability of a system to run multiple processors in parallel, where each processor can run one or more threads.",
        ],
    }
)

Create a simple OpenAI model that asks GPT to answer the question in two sentences. Call `mlflow.evaluate()` with the model and evaluation dataframe.

In [6]:
mlflow.set_experiment("openai-notebook")

with mlflow.start_run() as run:
    system_prompt = "Answer the following question in two sentences"
    basic_qa_model = mlflow.openai.log_model(
        model="gpt-4.1-mini",
        task=openai.chat.completions,
        name="model",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "{question}"},
        ],
    )
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        targets="ground_truth",  # specify which column corresponds to the expected output
        model_type="question-answering",  # model type indicates which metrics are relevant for this task
        evaluators="default",
    )
results.metrics

/Users/asfand/Documents/psrc/hwtt/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025/07/04 18:18:36 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-1a798636bdab4b8d819ed582a8f507ab
2025/07/04 18:18:36 INFO mlflow.tracking.fluent: Use `mlflow.set_active_model` to set the active model to a different one if needed.
2025/07/04 18:18:36 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-1a798636bdab4b8d819ed582a8f507ab
2025/07/04 18:18:36 INFO mlflow.tracking.fluent: Use `mlflow.set_active_model` to set the active model to a different one if needed.
2025/07/04 18:18:36 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call 

🏃 View run loud-ant-752 at: http://localhost:5001/#/experiments/2/runs/6c052e39b94a4d358beb7931ae6a29bc
🧪 View experiment at: http://localhost:5001/#/experiments/2


{'exact_match/v1': 0.0}

[Trace(trace_id=e8b1ae64d9fe4dbd819b184343563628), Trace(trace_id=5d4c40b5886c440c9b405b456915b545), Trace(trace_id=bad53a3b6c8948cbb2a00bc0b65a502f), Trace(trace_id=5b2be73939ae4b5ba4d3a3dad8d0ff6c)]

Inspect the evaluation results table as a dataframe to see row-by-row metrics to further assess model performance

In [7]:
results.tables["eval_results_table"]

inputs  \
0                         How does useEffect() work?   
1   What does the static keyword in a function mean?   
2        What does the 'finally' block in Python do?   
3  What is the difference between multiprocessing...   

                                        ground_truth  \
0  The useEffect() hook tells React that your com...   
1  Static members belongs to the class, rather th...   
2  'Finally' defines a block of code to run when ...   
3  Multithreading refers to the ability of a proc...   

                                             outputs  token_count  
0  useEffect() is a React Hook that runs side eff...           59  
1  The `static` keyword in a function limits the ...           59  
2  The 'finally' block in Python contains code th...           46  
3  Multiprocessing involves using multiple CPU co...           79

## LLM-judged correctness with OpenAI GPT-4

In [8]:
from mlflow.metrics.genai import EvaluationExample, answer_similarity

# Create an example to describe what answer_similarity means like for this problem.
example = EvaluationExample(
    input="What is MLflow?",
    output="MLflow is an open-source platform for managing machine "
           "learning workflows, including experiment tracking, model packaging, "
           "versioning, and deployment, simplifying the ML lifecycle.",
    score=4,
    justification="The definition effectively explains what MLflow is "
                  "its purpose, and its developer. It could be more concise for a 5-score.",
    grading_context={
        "targets": "MLflow is an open-source platform for managing "
                   "the end-to-end machine learning (ML) lifecycle. It was developed by Databricks, "
                   "a company that specializes in big data and machine learning solutions. MLflow is "
                   "designed to address the challenges that data scientists and machine learning "
                   "engineers face when developing, training, and deploying machine learning models."
    },
)

# Construct the metric using an OpenAI model as the judge
answer_similarity_metric = answer_similarity(model="openai:/gpt-4.1", examples=[example])

print(answer_similarity_metric)

EvaluationMetric(name=answer_similarity, greater_is_better=True, long_name=answer_similarity, version=v1, metric_details=
Task:
You must return the following fields in your response in two lines, one below the other:
score: Your numerical score for the model's answer_similarity based on the rubric
justification: Your reasoning about the model's answer_similarity score

You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called answer_similarity based on the input and output.
A definition of answer_similarity and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them be

Call `mlflow.evaluate()` again but with your new `answer_similarity_metric`

In [16]:
with mlflow.start_run() as run:
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        targets="ground_truth",
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[answer_similarity_metric],  # use the answer similarity metric created above
    )
results.metrics

2025/07/04 18:31:22 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-1a798636bdab4b8d819ed582a8f507ab
2025/07/04 18:31:22 INFO mlflow.tracking.fluent: Use `mlflow.set_active_model` to set the active model to a different one if needed.
2025/07/04 18:31:22 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
2025/07/04 18:31:25 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/07/04 18:31:26 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: NameError("name 'torch' is not defined")), skipping metric logging.
2025/07/04 18:31:26 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'toxicity' at index 1 in the `extra_metrics` parameter because it returned None.
100%|██████████| 1/1 [00:01<00:00,  1.61s/it]
2025/07/04 18:31:30 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: NameError("name 'torch' is not defined")), sk

🏃 View run capable-snail-947 at: http://localhost:5001/#/experiments/2/runs/42dac84a702c498b86d08a53d4aef274
🧪 View experiment at: http://localhost:5001/#/experiments/2


{'flesch_kincaid_grade_level/v1/mean': np.float64(15.371071124437997),
 'flesch_kincaid_grade_level/v1/variance': np.float64(7.428522898575418),
 'flesch_kincaid_grade_level/v1/p90': np.float64(17.8381550802139),
 'ari_grade_level/v1/mean': np.float64(17.383569756868518),
 'ari_grade_level/v1/variance': np.float64(9.656302602178414),
 'ari_grade_level/v1/p90': np.float64(20.28081114551083),
 'exact_match/v1': 0.0,
 'answer_similarity/v1/mean': np.float64(4.0),
 'answer_similarity/v1/variance': np.float64(1.5),
 'answer_similarity/v1/p90': np.float64(5.0)}

[Trace(trace_id=d5b3bbc744ad42d9877ba4c2c0918ad1), Trace(trace_id=b68e86f17e3e4d6c935a49c07d51628a), Trace(trace_id=c7b9382ca67c4bbb97128c503a01234b), Trace(trace_id=d550acf00d0448a9a022fb69d6991fcb)]

See the row-by-row LLM-judged answer similarity score and justifications

In [ ]:
results.tables["eval_results_table"]

## Custom LLM-judged metric for professionalism

Create a custom metric that will be used to determine professionalism of the model outputs. Use `make_genai_metric` with a metric definition, grading prompt, grading example, and judge model configuration

In [17]:
from mlflow.metrics.genai import EvaluationExample, make_genai_metric

professionalism_metric = make_genai_metric(
    name="professionalism",
    definition=(
        "Professionalism refers to the use of a formal, respectful, and appropriate style of communication that is tailored to the context and audience. It often involves avoiding overly casual language, slang, or colloquialisms, and instead using clear, concise, and respectful language"
    ),
    grading_prompt=(
        "Professionalism: If the answer is written using a professional tone, below "
        "are the details for different scores: "
        "- Score 1: Language is extremely casual, informal, and may include slang or colloquialisms. Not suitable for professional contexts."
        "- Score 2: Language is casual but generally respectful and avoids strong informality or slang. Acceptable in some informal professional settings."
        "- Score 3: Language is balanced and avoids extreme informality or formality. Suitable for most professional contexts. "
        "- Score 4: Language is noticeably formal, respectful, and avoids casual elements. Appropriate for business or academic settings. "
        "- Score 5: Language is excessively formal, respectful, and avoids casual elements. Appropriate for the most formal settings such as textbooks. "
    ),
    examples=[
        EvaluationExample(
            input="What is MLflow?",
            output=(
                "MLflow is like your friendly neighborhood toolkit for managing your machine learning projects. It helps you track experiments, package your code and models, and collaborate with your team, making the whole ML workflow smoother. It's like your Swiss Army knife for machine learning!"
            ),
            score=2,
            justification=(
                "The response is written in a casual tone. It uses contractions, filler words such as 'like', and exclamation points, which make it sound less professional. "
            ),
        )
    ],
    version="v1",
    model="openai:/gpt-4.1-mini",
    parameters={"temperature": 0.0},
    grading_context_columns=[],
    aggregations=["mean", "variance", "p90"],
    greater_is_better=True,
)

print(professionalism_metric)

EvaluationMetric(name=professionalism, greater_is_better=True, long_name=professionalism, version=v1, metric_details=
Task:
You must return the following fields in your response in two lines, one below the other:
score: Your numerical score for the model's professionalism based on the rubric
justification: Your reasoning about the model's professionalism score

You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called professionalism based on the input and output.
A definition of professionalism and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them before complet

Call `mlflow.evaluate` with your new professionalism metric. 

In [ ]:
with mlflow.start_run() as run:
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[professionalism_metric],  # use the professionalism metric we created above
    )
print(results.metrics)

In [ ]:
results.tables["eval_results_table"]

Lets see if we can improve `basic_qa_model` by creating a new model that could perform better by changing the system prompt.

Call `mlflow.evaluate()` using the new model. Observe that the professionalism score has increased!

In [ ]:
with mlflow.start_run() as run:
    system_prompt = "Answer the following question using extreme formality."
    professional_qa_model = mlflow.openai.log_model(
        model="gpt-4o-mini",
        task=openai.chat.completions,
        name="model",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "{question}"},
        ],
    )
    results = mlflow.evaluate(
        professional_qa_model.model_uri,
        eval_df,
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[professionalism_metric],
    )
print(results.metrics)

In [ ]:
results.tables["eval_results_table"]